In [124]:
import qutip as qt
import numpy as np
import sympy as sp
from qiskit import QuantumCircuit

rho_0 = sp.Matrix([[1,0],[0,0]])
rho_1 = sp.Matrix([[0,0],[0,1]])

ket_0 = sp.Matrix([1,0])
ket_1 = sp.Matrix([0,1])

H = 1/sp.sqrt(2)*sp.Matrix([[1,1],[1,-1]])


In [125]:
I = sp.Matrix([[1, 0], [0, 1]])
X = sp.Matrix([[0, 1], [1, 0]])
Y = sp.Matrix([[0, -sp.I], [sp.I, 0]])
Z = sp.Matrix([[1, 0], [0, -1]])

theta, phi, p = sp.symbols('theta phi p', real=True)
a = sp.cos(theta / 2)
b = sp.exp(sp.I * phi) * sp.sin(theta / 2)
a_bar = sp.conjugate(a)
b_bar = sp.conjugate(b)

psi = a*ket_0 + b*ket_1

psi_conj = psi.conjugate()
psi_dg = psi_conj.transpose()

rho_a = psi*psi_dg


In [126]:
rho = sp.kronecker_product(rho_a,rho_0,rho_0,rho_0,rho_0,rho_0,rho_0,rho_0,rho_0)

In [127]:
K_0 = sp.sqrt(p)*sp.kronecker_product(Z@X,Z@X,Z@X,Z@X,Z@X,Z@X,Z@X,Z@X,Z@X)
K_1 = sp.sqrt(1-p)*sp.kronecker_product(I,I,I,I,I,I,I,I,I)


In [128]:
U_1 = sp.kronecker_product(rho_0,I,I,I,I,I,I,I,I) + sp.kronecker_product(rho_1,I,I,X,I,I,I,I,I)
U_2 = sp.kronecker_product(rho_0,I,I,I,I,I,I,I,I) + sp.kronecker_product(rho_1,I,I,I,I,I,X,I,I)
rho_H = sp.kronecker_product(H,I,I,H,I,I,H,I,I)
U_3 = (sp.kronecker_product(rho_0,I,I,I,I,I,I,I,I) + sp.kronecker_product(rho_1,X,I,I,I,I,I,I,I))*(sp.kronecker_product(I,I,I,rho_0,I,I,I,I,I) + sp.kronecker_product(I,I,I,rho_1,X,I,I,I,I))*(sp.kronecker_product(I,I,I,I,I,I,rho_0,I,I) + sp.kronecker_product(I,I,I,I,I,I,rho_1,X,I))
U_4 = (sp.kronecker_product(rho_0,I,I,I,I,I,I,I,I) + sp.kronecker_product(rho_1,I,X,I,I,I,I,I,I))*(sp.kronecker_product(I,I,I,rho_0,I,I,I,I,I) + sp.kronecker_product(I,I,I,rho_1,I,X,I,I,I))*(sp.kronecker_product(I,I,I,I,I,I,rho_0,I,I) + sp.kronecker_product(I,I,I,I,I,I,rho_1,I,X))
U_6 = U_3
U_7 = U_4
U_8 = sp.kronecker_product(I,rho_0,rho_0,I,I,I,I,I,I) + sp.kronecker_product(I,rho_0,rho_1,I,I,I,I,I,I) + sp.kronecker_product(I,rho_1,rho_0,I,I,I,I,I,I) + sp.kronecker_product(X,rho_1,rho_1,I,I,I,I,I,I) 
U_8 = U_8*(sp.kronecker_product(I,I,I,I,rho_0,rho_0,I,I,I) + sp.kronecker_product(I,I,I,I,rho_0,rho_1,I,I,I) + sp.kronecker_product(I,I,I,I,rho_1,rho_0,I,I,I) + sp.kronecker_product(I,I,I,X,rho_1,rho_1,I,I,I))
U_8 = U_8*(sp.kronecker_product(I,I,I,I,I,I,I,rho_0,rho_0) + sp.kronecker_product(I,I,I,I,I,I,I,rho_0,rho_1) + sp.kronecker_product(I,I,I,I,I,I,I,rho_1,rho_0) + sp.kronecker_product(I,I,I,I,I,I,X,rho_1,rho_1))
U_9 = U_1
U_10 = U_2
U_11 = sp.kronecker_product(I,I,I,rho_0,I,I,rho_0,I,I) + sp.kronecker_product(I,I,I,rho_0,I,I,rho_1,I,I) + sp.kronecker_product(I,I,I,rho_1,I,I,rho_0,I,I) + sp.kronecker_product(X,I,I,rho_1,I,I,rho_1,I,I) 


In [129]:
Ua = U_11*U_10*U_9*rho_H*U_8*U_7*U_6*K_0*U_4*U_3*rho_H*U_2*U_1
Ub = U_11*U_10*U_9*rho_H*U_8*U_7*U_6*K_1*U_4*U_3*rho_H*U_2*U_1


In [130]:
final = Ub@rho@Ub.transpose() + Ua@rho@Ua.transpose()


In [136]:
import sympy as sp
import itertools

def partial_trace(rho, n, qubits_to_trace_out):
    dim = 2 ** n
    
    # Define the indices for the new matrix
    remaining_qubits = sorted(set(range(n)) - set(qubits_to_trace_out))
    remaining_dim = 2 ** len(remaining_qubits)
    
    # Initialize the reduced density matrix
    reduced_rho = sp.zeros(remaining_dim, remaining_dim)
    
    # Iterate over all possible states of the remaining qubits
    for idx1, idx2 in itertools.product(range(remaining_dim), repeat=2):
        idx1_bin = format(idx1, f'0{len(remaining_qubits)}b')
        idx2_bin = format(idx2, f'0{len(remaining_qubits)}b')
        
        total_sum = 0
        for trace_state in range(2 ** len(qubits_to_trace_out)):
            trace_state_bin = format(trace_state, f'0{len(qubits_to_trace_out)}b')
            
            # Construct the full state for the original system
            full_state1 = ['0'] * n
            full_state2 = ['0'] * n
            
            for i, q in enumerate(remaining_qubits):
                full_state1[q] = idx1_bin[i]
                full_state2[q] = idx2_bin[i]
            
            for i, q in enumerate(qubits_to_trace_out):
                full_state1[q] = trace_state_bin[i]
                full_state2[q] = trace_state_bin[i]
            
            # Convert binary strings to indices
            full_index1 = int(''.join(full_state1), 2)
            full_index2 = int(''.join(full_state2), 2)
            
            total_sum += rho[full_index1, full_index2]
        
        reduced_rho[idx1, idx2] = total_sum
    
    return reduced_rho


In [137]:
def trace_each_qubit(final_state, total_qubits):
    for qubit in range(total_qubits):
        qubits_to_trace_out = [i for i in range(total_qubits) if i != qubit]

        reduced_rho = partial_trace(final_state, total_qubits, qubits_to_trace_out)
        
        reduced_rho = sp.simplify(reduced_rho)

        print("Reduced density matrix for qubit: ",  {qubit}, '\n', {reduced_rho}, '\n')

n = 9
trace_each_qubit(final, n)


Reduced density matrix for qubit:  {0} 
 {Matrix([
[          -2*p*cos(theta/2)**2 + p + cos(theta/2)**2, (-p*exp(2*I*phi) - p + 1)*exp(-I*phi)*sin(theta)/2],
[(-p + (1 - p)*exp(2*I*phi))*exp(-I*phi)*sin(theta)/2,         -2*p*sin(theta/2)**2 + p + sin(theta/2)**2]])} 

Reduced density matrix for qubit:  {1} 
 {Matrix([
[1, 0],
[0, 0]])} 

Reduced density matrix for qubit:  {2} 
 {Matrix([
[1, 0],
[0, 0]])} 

Reduced density matrix for qubit:  {3} 
 {Matrix([
[1, 0],
[0, 0]])} 

Reduced density matrix for qubit:  {4} 
 {Matrix([
[1, 0],
[0, 0]])} 

Reduced density matrix for qubit:  {5} 
 {Matrix([
[1, 0],
[0, 0]])} 

Reduced density matrix for qubit:  {6} 
 {Matrix([
[1, 0],
[0, 0]])} 

Reduced density matrix for qubit:  {7} 
 {Matrix([
[1, 0],
[0, 0]])} 

Reduced density matrix for qubit:  {8} 
 {Matrix([
[1, 0],
[0, 0]])} 



In [139]:
n=9
qubits_to_trace_out = [1,2,3,4,5,6,7,8]

# Compute the reduced density matrix
reduced_rho = partial_trace(final, n, qubits_to_trace_out)

reduced_rho = sp.simplify(reduced_rho)
reduced_rho

Matrix([
[          -2*p*cos(theta/2)**2 + p + cos(theta/2)**2, (-p*exp(2*I*phi) - p + 1)*exp(-I*phi)*sin(theta)/2],
[(-p + (1 - p)*exp(2*I*phi))*exp(-I*phi)*sin(theta)/2,         -2*p*sin(theta/2)**2 + p + sin(theta/2)**2]])

In [140]:
fidelity = psi_dg*reduced_rho*psi

sp.simplify(fidelity)

Matrix([[-p*exp(2*I*phi)*sin(theta)**2/4 + p*sin(theta)**2/2 - p - p*exp(-2*I*phi)*sin(theta)**2/4 + 1]])

In [141]:

integral = sp.integrate(fidelity * sp.sin(theta), (theta, 0, sp.pi), (phi, 0, 2*sp.pi), (p, 0, 1))

normalized_integral = integral / (4 * sp.pi)
normalized_integral.simplify()


Matrix([[2/3]])